### Convolutional Neural Networks

In [33]:
import torch
import torchvision
from torchsummary import summary

In [48]:
class LeNet(torch.nn.Module):

    def __init__(self, num_classes):
        super().__init__()
        self.ConvLayer1 = torch.nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding='same')
        self.PoolLayer = torch.nn.AvgPool2d(kernel_size=2)
        self.ConvLayer2 = torch.nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.ConvLayer3 = torch.nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5)
        self.Linear1 = torch.nn.Linear(in_features=120, out_features=84)
        self.Linear2 = torch.nn.Linear(in_features=84, out_features=num_classes)
    
    def forward(self, x):
        x = torch.nn.functional.tanh(self.ConvLayer1(x))
        x = self.PoolLayer(x)
        x = torch.nn.functional.tanh(self.ConvLayer2(x))
        x = self.PoolLayer(x)
        x = torch.nn.functional.tanh(self.ConvLayer3(x))
        x = torch.nn.Flatten()(x)
        x = self.Linear1(x)
        x = torch.nn.functional.tanh(x)
        outputs = self.Linear2(x)

        return outputs

In [36]:
def train(model, trainLoader, criterion, optimizer, device):
    model.train()
    trainLoss = []
    correct, total = 0, 0

    for (inputs, labels) in trainLoader:

        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        trainLoss.append(loss.item())

        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)
        loss.backward()
        optimizer.step()
    
    epochAccuracy = 100 * correct / total
    epochLoss = sum(trainLoss)/len(trainLoss)

    return epochAccuracy, epochLoss

In [37]:
def evaluateModel(model, validLoader, criterion, device):
    model.eval()
    losses = []
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in validLoader:

            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            losses.append(loss)

            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)
    
    epoch_acc = 100 * correct / total
    epoch_loss = sum(losses)/len(losses)

    return epoch_acc, epoch_loss

In [42]:
train_data = torchvision.datasets.MNIST(root='data', download=False, train=True)
test_data = torchvision.datasets.MNIST(root='data', download=False, train=False)

In [43]:
VALID_RATIO = 0.9
number_train_data = int(len(train_data)*0.9)
number_valid_samples = len(train_data) - number_train_data

train_data, valid_data = torch.utils.data.random_split(train_data,
                                                       [number_train_data, number_valid_samples])

In [44]:
mean = train_data.dataset.data.float().mean() / 255
std = train_data.dataset.data.float().std() / 255

transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                            torchvision.transforms.Normalize(mean=[mean], std=[std])])

train_data.dataset.transform = transform
valid_data.dataset.transform = transform

BATCH_SIZE = 256
trainLoader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE,
                                          shuffle=True)

validLoader = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE)

In [57]:
print(mean, std)

tensor(0.1307) tensor(0.3081)


In [52]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

num_classes = len(train_data.dataset.classes)

leNetClassifier = LeNet(num_classes)
leNetClassifier.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(leNetClassifier.parameters(), lr=1e-4)

num_epochs = 20
model_path = './model'

trainLosses, trainAccuracies = [], []
validLosses, validAccuracies = [], []
best_loss_eval = 100

for epoch in range(num_epochs):

    trainAccuracy, trainLoss = train(leNetClassifier, trainLoader, criterion, optimizer, device)
    trainAccuracies.append(trainAccuracy)
    trainLosses.append(trainLoss)

    validAccuracy, validLoss = evaluateModel(leNetClassifier, validLoader, criterion, device)
    validAccuracies.append(validAccuracy)
    validLosses.append(validLoss)

    if validLoss < best_loss_eval:
        torch.save(leNetClassifier.state_dict(), model_path+'/leNet.pt')
    
    print(f"End of epoch {epoch+1}, Accuracy: {trainAccuracy:.2f}%, Loss: {trainLoss:.4f}, Accuracy: {validAccuracy:.2f}%, Loss: {validLoss:.4f}")

    leNetClassifier.load_state_dict(torch.load(model_path+'/leNet.pt'))
    leNetClassifier.eval()

End of epoch 1, Accuracy: 58.70%, Loss: 1.5854, Accuracy: 80.45%, Loss: 0.8990
End of epoch 2, Accuracy: 84.08%, Loss: 0.6834, Accuracy: 88.10%, Loss: 0.4997
End of epoch 3, Accuracy: 88.89%, Loss: 0.4434, Accuracy: 90.83%, Loss: 0.3563
End of epoch 4, Accuracy: 91.05%, Loss: 0.3390, Accuracy: 92.50%, Loss: 0.2821
End of epoch 5, Accuracy: 92.37%, Loss: 0.2774, Accuracy: 93.72%, Loss: 0.2331
End of epoch 6, Accuracy: 93.58%, Loss: 0.2318, Accuracy: 94.78%, Loss: 0.1963
End of epoch 7, Accuracy: 94.53%, Loss: 0.1957, Accuracy: 95.27%, Loss: 0.1669
End of epoch 8, Accuracy: 95.36%, Loss: 0.1664, Accuracy: 95.95%, Loss: 0.1434
End of epoch 9, Accuracy: 95.98%, Loss: 0.1442, Accuracy: 96.45%, Loss: 0.1252
End of epoch 10, Accuracy: 96.46%, Loss: 0.1268, Accuracy: 96.85%, Loss: 0.1107
End of epoch 11, Accuracy: 96.82%, Loss: 0.1131, Accuracy: 96.98%, Loss: 0.1005
End of epoch 12, Accuracy: 97.05%, Loss: 0.1025, Accuracy: 97.20%, Loss: 0.0915
End of epoch 13, Accuracy: 97.33%, Loss: 0.0936, 

In [56]:
test_data.transform = transform
testLoader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE)

testAccuracy, testLoss = evaluateModel(leNetClassifier, testLoader, criterion, device)
print(testAccuracy, testLoss)

98.2 tensor(0.0560, device='cuda:0')
